In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import glob
import re
import fileinput
import csv
import ast
import nltk
from nltk import word_tokenize
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from xml.etree import ElementTree as ET
import flair
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from typing import List
from flair.data import Sentence
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mchou001/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
2023-01-20 11:13:17.582360: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [3]:
pwd

'/home/mchou001/university-classifier'

In [4]:
text_path = '/home/mchou001/university-classifier/data/clean_data/*.txt'
ann_path = '/home/mchou001/university-classifier/data/annotated_data/*.xml'
clean_files = sorted(glob.glob(text_path), key=numericalSort)
ann_files = sorted(glob.glob(ann_path), key=numericalSort)

### Processing Text files

In [5]:
## reading text files and converting it single csv file
text_list = []
for path in clean_files:
    with open(path) as source:
        lines = source.readlines()
        text_list.append(lines)
        result = [list(item) for item in list(zip(text_list))]
        dataframe = pd.DataFrame(result, columns = ['text'])
        dataframe.to_csv('./result/clean_text.csv', index = False)

In [6]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [7]:
##further processing text data
text_data = pd.read_csv('./result/clean_text.csv')
list_data = []
for texts in text_data['text']:
    etd_meta = ast.literal_eval(str(texts))
    etd_text = [items for items in etd_meta]
    meta_text_list = flatten_list(etd_text)
    list_data.append(meta_text_list)

data = zip(list_data)
dataframe = pd.DataFrame(data)

list_text = []
for items in dataframe[0]:
    vals = [ ' '.join(element.lstrip().rstrip().split('  ')) for element in items]
    vals_str = ' '.join(vals)
    list_value = [vals_str]
    list_text.append(list_value)

data2 = zip(list_text)
dataframe2 = pd.DataFrame(data2, columns = ['text'])
dataframe2.to_csv('./result/clean_text-final.csv', index = False)

### Processing Annotating Files

In [8]:
def append_ann(files):
    xml_files = sorted(glob.glob(files), key=numericalSort)
    new_data = b""
    for xml_file in xml_files:
        data = ET.parse(xml_file).getroot()
        temp = ET.tostring(data)
        new_data = new_data+temp
    return new_data

In [9]:
def remove_punct(withpunct):
    punctuations = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
    without_punct = ""
    char = 'nan'
    for char in withpunct:
        if char not in punctuations:
            without_punct = without_punct + char
    return(without_punct)

In [10]:
def BIO_TAG(files):

    docs = []
    sents = []

    for d in soup.find_all("document"):
        for word in d:
            tags = []
            NoneType = type(None)
            other_tag = 'O'
            if isinstance(word.name, NoneType) == True:
                withoutpunct = remove_punct(word)
                temp = withoutpunct.split()
                for token in temp:
                    tags.append((token, other_tag))
            else:
                prev_tag = other_tag
                withoutpunct = remove_punct(word)
                temp = withoutpunct.split()
                for token in temp:
                    #beginning of the token
                    if tags != 'O' and prev_tag == 'O' :
                        tags.append((token, "B-"+word.name)) 
                        tag = "B-"+word.name
                        prev_tag = tag
                    #inside of the token
                    elif prev_tag != 'O' and prev_tag == tag:
                        tags.append((token, "I-"+word.name))
                        tag = "I-"+word.name
                        prev_tag = tag
                    #adjacent of the token
                    elif prev_tag != 'O' and prev_tag != tag:
                        tags.append((token, "B-"+word.name))
                        tag = "B-"+word.name
                        prev_tag = tag

            sents = sents + tags

    docs.append(sents)
    return docs

In [11]:
allxmlfiles = append_ann(ann_path)
soup = BeautifulSoup(allxmlfiles, "html.parser")
ner_tagging = BIO_TAG(soup)

In [12]:
# fp = open("./result/train.txt", 'w')
# for i in ner_tagging:
#     for j in i:
#         fp.write('\t'.join(list(j))+'\n')
# fp.close()

In [17]:
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '/home/mchou001/university-classifier/result/'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'dev.txt')

tag_dictionary = corpus.make_label_dictionary(label_type='ner')
#print(tag_dictionary)

2023-01-18 01:44:55,069 Reading data from /home/mchou001/university-classifier/result
2023-01-18 01:44:55,070 Train: /home/mchou001/university-classifier/result/train.txt
2023-01-18 01:44:55,070 Dev: /home/mchou001/university-classifier/result/dev.txt
2023-01-18 01:44:55,071 Test: /home/mchou001/university-classifier/result/test.txt
2023-01-18 01:44:55,907 Computing label dictionary. Progress:


343it [00:00, 16313.96it/s]

2023-01-18 01:44:55,932 Dictionary created for label 'ner' with 8 values: author (seen 347 times), university (seen 345 times), title (seen 344 times), degree (seen 335 times), year (seen 306 times), program (seen 257 times), advisor (seen 72 times)
Dictionary with 8 tags: <unk>, author, university, title, degree, year, program, advisor


In [23]:
print(len(corpus.train))
print(corpus.train[0].to_tagged_string('ner'))

343
Sentence: "SOME ASPECTS OF RADIATION INDUCED NUCLEATION IN WATER by Chih-Ping Tso B.Tech. Loughborough University of Technology United Kingdom 1968 Submitted in Partial Fulfillment of the Requirements for the Degree of Master of Science at the Massachusetts Institute of Technology August 1970 Signature of Author Department of Nuclear Engineering Certified by Thesis Supervisor Accepted by Chairman Departmental Committee Archives on Graduate Students MASS. INST. TECH. SEP 21 1970 LIBRARIES" → ["SOME ASPECTS OF RADIATION INDUCED NUCLEATION IN WATER"/title, "Chih-Ping Tso"/author, "Master of Science"/degree, "Massachusetts Institute of Technology"/university, "1970"/year, "Nuclear Engineering"/program]


### Training

In [18]:
## With Stacked embeddings which combines classic word embeddings and flair embeddings
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

2023-01-18 01:46:41,746 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpijnel1j4


100%|██████████| 160000128/160000128 [00:09<00:00, 16364833.17B/s]

2023-01-18 01:46:51,902 copying /tmp/tmpijnel1j4 to cache at /home/mchou001/.flair/embeddings/glove.gensim.vectors.npy


2023-01-18 01:46:53,432 removing temp file /tmp/tmpijnel1j4
2023-01-18 01:46:53,834 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpmicf7338


100%|██████████| 21494764/21494764 [00:01<00:00, 11326360.37B/s]

...

2023-01-18 01:47:07,762 copying /tmp/tmpk1433n35 to cache at /home/mchou001/.flair/embeddings/news-forward-0.4.1.pt


2023-01-18 01:47:08,470 removing temp file /tmp/tmpk1433n35
2023-01-18 01:47:12,363 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpblvfswvw


100%|██████████| 73034575/73034575 [00:04<00:00, 15330505.55B/s]

2023-01-18 01:47:17,507 copying /tmp/tmpblvfswvw to cache at /home/mchou001/.flair/embeddings/news-backward-0.4.1.pt


2023-01-18 01:47:18,211 removing temp file /tmp/tmpblvfswvw


In [19]:
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

2023-01-18 01:48:27,259 SequenceTagger predicts: Dictionary with 29 tags: O, S-author, B-author, E-author, I-author, S-university, B-university, E-university, I-university, S-title, B-title, E-title, I-title, S-degree, B-degree, E-degree, I-degree, S-year, B-year, E-year, I-year, S-program, B-program, E-program, I-program, S-advisor, B-advisor, E-advisor, I-advisor


In [20]:
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

In [22]:
trainer.train('/home/mchou001/university-classifier',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50,
              embeddings_storage_mode='gpu')

2023-01-18 01:52:48,303 ----------------------------------------------------------------------------------------------------
2023-01-18 01:52:48,304 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, b

100%|██████████| 4/4 [00:01<00:00,  2.35it/s]

2023-01-18 01:52:54,246 Evaluating as a multi-label problem: False
2023-01-18 01:52:54,266 DEV : loss 1.6160402297973633 - f1-score (micro avg)  0.0238
2023-01-18 01:52:54,272 BAD EPOCHS (no improvement): 0
2023-01-18 01:52:54,275 saving best model


2023-01-18 01:52:59,245 ----------------------------------------------------------------------------------------------------
2023-01-18 01:52:59,478 epoch 2 - iter 1/11 - loss 1.62344853 - samples/sec: 138.20 - lr: 0.100000
2023-01-18 01:52:59,620 epoch 2 - iter 2/11 - loss 1.75934599 - samples/sec: 226.87 - lr: 0.100000
2023-01-18 01:52:59,753 epoch 2 - iter 3/11 - loss 1.73614457 - samples/sec: 243.22 - lr: 0.100000
2023-01-18 01:52:59,885 epoch 2 - iter 4/11 - loss 1.75075312 - samples/sec: 244.34 - lr: 0.100000
2023-01-18 01:53:00,042 epoch 2 - iter 5/11 - loss 1.72601389 - samples/sec: 204.86 - lr: 0.100000
2023-01-18 01:53:00,191 epoch 2 - iter 6/11 - loss 1.73014461 - samples/sec: 217.02 - lr: 0.100000
2023-01-18 01:53:00,391 epoch 2 - iter 7/11 - loss 1.69781881 - samples/sec: 160.74 - lr: 0.100000
2023-01-18 01:53:00,530 epoch 2 - iter 8/11 - loss 1.70842722 - samples/sec: 231.76 - lr: 0.100000
2023-01-18 01:53:00,685 epoch 2 - iter 9/11 - loss 1.70134159 - samples/sec: 208.48

100%|██████████| 4/4 [00:00<00:00,  4.98it/s]

...

2023-01-18 01:57:11,375 Evaluating as a multi-label problem: False
2023-01-18 01:57:11,392 DEV : loss 0.05114128068089485 - f1-score (micro avg)  0.8788
2023-01-18 01:57:11,398 BAD EPOCHS (no improvement): 2


2023-01-18 01:57:16,439 ----------------------------------------------------------------------------------------------------
2023-01-18 01:57:16,441 loading file /home/mchou001/university-classifier/best-model.pt
2023-01-18 01:57:18,141 SequenceTagger predicts: Dictionary with 31 tags: O, S-author, B-author, E-author, I-author, S-university, B-university, E-university, I-university, S-title, B-title, E-title, I-title, S-degree, B-degree, E-degree, I-degree, S-year, B-year, E-year, I-year, S-program, B-program, E-program, I-program, S-advisor, B-advisor, E-advisor, I-advisor, <START>, <STOP>


100%|██████████| 2/2 [00:00<00:00,  2.94it/s]

2023-01-18 01:57:19,242 Evaluating as a multi-label problem: False
2023-01-18 01:57:19,255 0.952	0.952	0.952	0.9084000000000001
2023-01-18 01:57:19,256 
Results:
- F-score (micro) 0.952
- F-score (macro) 0.952
- Accuracy 0.9084000000000001

By class:
              precision    recall  f1-score   support

       title     1.0000    1.0000    1.0000        50
      author     0.9800    0.9800    0.9800        50
      degree     1.0000    1.0000    1.0000        50
  university     1.0000    1.0000    1.0000        50
        year     0.7800    0.7800    0.7800        50

   micro avg     0.9520    0.9520    0.9520       250
   macro avg     0.9520    0.9520    0.9520       250
weighted avg     0.9520    0.9520    0.9520       250

2023-01-18 01:57:19,256 ----------------------------------------------------------------------------------------------------


{'test_score': 0.952,
 'dev_score_history': [0.0237741456166419,
  0.10559006211180123,
  0.04722222222222222,
  0.10377358490566038,
  0.19235836627140976,
  0.28502415458937197,
  0.3662280701754386,
  0.49333333333333335,
  0.5475935828877005,
  0.689419795221843,
  0.6628194558944766,
  0.7422334172963896,
  0.7532244196044712,
  0.7644067796610169,
  0.802415875754961,
  0.7495291902071562,
  0.8044982698961938,
  0.8018099547511313,
  0.8195876288659794,
  0.7285342584562011,
  0.8206039076376553,
  0.8364279398762158,
  0.8311229000884173,
  0.8263888888888888,
  0.8136200716845879,
  0.8452696728558797,
  0.863716814159292,
  0.8549488054607508,
  0.841927303465765,
  0.8374384236453203,
  0.8472573839662447,
  0.8780487804878049,
  0.8736842105263158,
  0.8641975308641976,
  0.8847845206684257,
  0.8855368234250223,
  0.8710247349823322,
  0.8748890860692102,
  0.8764444444444445,
  0.8733392382639505,
  0.8665480427046264,
  0.8760035682426404,
  0.8758865248226951,
  0.87344

## Using the saved model we can run inference

In [3]:
# load the trained model
model = SequenceTagger.load('/home/mchou001/university-classifier/final-model.pt')
# create example sentence
text = Sentence('PSU')
# predict the tags
model.predict(text)
print(text.labels)

2023-01-20 11:13:34,040 loading file /home/mchou001/university-classifier/final-model.pt
2023-01-20 11:13:52,002 SequenceTagger predicts: Dictionary with 31 tags: O, S-author, B-author, E-author, I-author, S-university, B-university, E-university, I-university, S-title, B-title, E-title, I-title, S-degree, B-degree, E-degree, I-degree, S-year, B-year, E-year, I-year, S-program, B-program, E-program, I-program, S-advisor, B-advisor, E-advisor, I-advisor, <START>, <STOP>
['Span[0:1]: "PSU"'/'title' (0.5597)]


# Prediction on 500 new ETD university field

## Preparing the labeled data for prediction

In [4]:
new_data = pd.read_csv("etd_records_v5.csv")
university = new_data['university']
uni_data = pd.DataFrame(university, columns = ['university'])
uni_text = uni_data.dropna().reset_index(drop=True)
uni_text

,university
0,"University of California, Berkeley"
1,"University of California, Berkeley"
2,"University of California, Berkeley"
3,"University of California, Berkeley"
4,"University of California, Berkeley"
...,...
492,Texas Tech University
493,Rice University
494,Texas Tech University
495,Texas Tech University


In [5]:
uni = [i for i in uni_text['university'] if len(i) <= 13]
uni

['Virginia Tech',
 'Virginia Tech',
 'Virginia Tech',
 'Virginia Tech',
 'Virginia Tech',
 'Virginia Tech']

In [6]:
uni_text['label'] = ['non_university' if x == 'Virginia Tech' else 'university' for x in uni_text['university']]

In [7]:
uni_text.to_csv("./result/university.csv", index=None)

In [8]:
university_text = pd.read_csv('./result/university.csv')
university_text['label'].to_frame().to_csv('./result/true_label.csv', index = None)

In [9]:
model = SequenceTagger.load('/home/mchou001/university-classifier/final-model.pt')
pred_labels = []
for uni in university_text['university']:
    text = Sentence(uni)
    model.predict(text)
    pred_labels.append([text])

2023-01-18 18:32:52,915 loading file /home/mchou001/university-classifier/final-model.pt
2023-01-18 18:32:54,658 SequenceTagger predicts: Dictionary with 31 tags: O, S-author, B-author, E-author, I-author, S-university, B-university, E-university, I-university, S-title, B-title, E-title, I-title, S-degree, B-degree, E-degree, I-degree, S-year, B-year, E-year, I-year, S-program, B-program, E-program, I-program, S-advisor, B-advisor, E-advisor, I-advisor, <START>, <STOP>


In [12]:
data_frame = pd.DataFrame(pred_labels, columns = ['pred_labels'])
data_frame.to_csv('./result/pred_labels.csv', index = None)

In [22]:
pred_label = []
pred = pd.read_csv('./result/pred_labels.csv')
for entity in pred['pred_labels']:
    pattern = re.search(r"/[?]*(\w+)", entity)
    text = pattern.group(1)
    pred_label.append(text)

In [24]:
pred_frame = pd.DataFrame(pred_label, columns = ['predicted_label'])
pred_frame.to_csv('./result/predicted_label.csv', index = None)

In [25]:
y_predicted_frame = pd.read_csv("./result/predicted_label.csv")
y_pred = []
for i in y_predicted_frame['predicted_label']:
    if i=='university':
        y_pred.append(1.0)
    else:
        y_test.append(0.0)

In [26]:
y_true_frame = pd.read_csv('./result/true_label.csv')
y_true = []
for i in y_true_frame['label']:
    if i=='university':
        y_true.append(1.0)
    else:
        y_true.append(0.0)

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print(accuracy_score(y_true, y_pred))

[[  0   6]
 [  0 491]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.99      1.00      0.99       491

    accuracy                           0.99       497
   macro avg       0.49      0.50      0.50       497
weighted avg       0.98      0.99      0.98       497

0.9879275653923542


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
